In [0]:
import json
import requests
from pyspark.sql.functions import *
from pyspark.sql.types import *

**Reading Mongo DB data**

In [0]:
uri = "mongodb+srv://<usuario>:<senha>@unicluster.ixhvw.mongodb.net/ibge.pnadc20203?retryWrites=true&w=majority"
 
ibge_mongoURL = spark.read.format("mongo")\
                  .option("uri", uri)\
                  .load()

In [0]:
ibge_mongoURL.printSchema()
# ibge_mongo.show(truncate=False)

root
-- _id: struct (nullable = true)
 |-- oid: string (nullable = true)
-- ano: integer (nullable = true)
-- anosesco: double (nullable = true)
-- cor: string (nullable = true)
-- graduacao: string (nullable = true)
-- horastrab: double (nullable = true)
-- idade: integer (nullable = true)
-- ocup: string (nullable = true)
-- renda: double (nullable = true)
-- sexo: string (nullable = true)
-- trab: string (nullable = true)
-- trimestre: integer (nullable = true)
-- uf: string (nullable = true)

**Defining the support functions that will parse the json object and transform it into a dataframe**

In [0]:
def convert_single_object_per_line(json_list):
    json_string = ""
    for line in json_list:
        json_string += json.dumps(line) + "\n"
    return json_string
 
def parse_dataframe(json_data):
    r = convert_single_object_per_line(json_data)
    mylist = []
    for line in r.splitlines():
        mylist.append(line)
    rdd = sc.parallelize(mylist)    
    df = spark.read.json(rdd)
    return df

**Creating the dataframe from the API**

In [0]:
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/regioes/1|2|3|4|5/municipios'
r =  requests.get(url)
j = r.json()
ibge_apiURL = parse_dataframe(j)

In [0]:
ibge_apiURL.printSchema()
ibge_apiURL.show(truncate=False)

root
-- id: long (nullable = true)
-- microrregiao: struct (nullable = true)
 |-- id: long (nullable = true)
 |-- mesorregiao: struct (nullable = true)
 | |-- UF: struct (nullable = true)
 | | |-- id: long (nullable = true)
 | | |-- nome: string (nullable = true)
 | | |-- regiao: struct (nullable = true)
 | | | |-- id: long (nullable = true)
 | | | |-- nome: string (nullable = true)
 | | | |-- sigla: string (nullable = true)
 | | |-- sigla: string (nullable = true)
 | |-- id: long (nullable = true)
 | |-- nome: string (nullable = true)
 |-- nome: string (nullable = true)
-- nome: string (nullable = true)
-- regiao-imediata: struct (nullable = true)
 |-- id: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- regiao-intermediaria: struct (nullable = true)
 | |-- UF: struct (nullable = true)
 | | |-- id: long (nullable = true)
 | | |-- nome: string (nullable = true)
 | | |-- regiao: struct (nullable = true)
 | | | |-- id: long (nullable = true)
 | | | |-- nome: string (nullable = true)
 | | | |-- sigla: string (nullable = true)
 | | |-- sigla: string (nullable = true)
 | |-- id: long (nullable = true)
 | |-- nome: string (nullable = true)

+-------+----------------------------------------------------------------------------------------+------------------------+-----------------------------------------------------------------------------+
id |microrregiao |nome |regiao-imediata |
+-------+----------------------------------------------------------------------------------------+------------------------+-----------------------------------------------------------------------------+
1100015|[11006, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Cacoal] |Alta Floresta D'Oeste |[110005, Cacoal, [[11, Rondônia, [1, Norte, N], RO], 1102, Ji-Paraná]] |
1100023|[11003, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Ariquemes] |Ariquemes |[110002, Ariquemes, [[11, Rondônia, [1, Norte, N], RO], 1101, Porto Velho]] |
1100031|[11008, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Colorado do Oeste]|Cabixi |[110006, Vilhena, [[11, Rondônia, [1, Norte, N], RO], 1102, Ji-Paraná]] |
1100049|[11006, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Cacoal] |Cacoal |[110005, Cacoal, [[11, Rondônia, [1, Norte, N], RO], 1102, Ji-Paraná]] |
1100056|[11008, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Colorado do Oeste]|Cerejeiras |[110006, Vilhena, [[11, Rondônia, [1, Norte, N], RO], 1102, Ji-Paraná]] |
1100064|[11008, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Colorado do Oeste]|Colorado do Oeste |[110006, Vilhena, [[11, Rondônia, [1, Norte, N], RO], 1102, Ji-Paraná]] |
1100072|[11008, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Colorado do Oeste]|Corumbiara |[110006, Vilhena, [[11, Rondônia, [1, Norte, N], RO], 1102, Ji-Paraná]] |
1100080|[11002, [[11, Rondônia, [1, Norte, N], RO], 1101, Madeira-Guaporé], Guajará-Mirim] |Costa Marques |[110004, Ji-Paraná, [[11, Rondônia, [1, Norte, N], RO], 1102, Ji-Paraná]] |
1100098|[11006, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Cacoal] |Espigão D'Oeste |[110005, Cacoal, [[11, Rondônia, [1, Norte, N], RO], 1102, Ji-Paraná]] |
1100106|[11002, [[11, Rondônia, [1, Norte, N], RO], 1101, Madeira-Guaporé], Guajará-Mirim] |Guajará-Mirim |[110001, Porto Velho, [[11, Rondônia, [1, Norte, N], RO], 1101, Porto Velho]]|
1100114|[11004, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Ji-Paraná] |Jaru |[110003, Jaru, [[11, Rondônia, [1, Norte, N], RO], 1101, Porto Velho]] |
1100122|[11004, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Ji-Paraná] |Ji-Paraná |[110004, Ji-Paraná, [[11, Rondônia, [1, Norte, N], RO], 1102, Ji-Paraná]] |
1100130|[11003, [[11, Rondônia, [1, Norte, N], RO], 1102, Leste Rondoniense], Ariquemes] |Machadinho D'Oeste |[110003, Jaru, [[11, Rondônia, [1, Norte, N], RO], 1101, Porto Velho]] |
1100148|[11005, [[11, Rondônia, [1, Norte, N],

In [0]:
ibge_mongoURL.write.parquet("/FileStore/ibge-mongo", mode="overwrite")
ibge_apiURL.write.parquet("/FileStore/ibge-api", mode="overwrite")

In [0]:
ibge_api = spark.read.parquet("/FileStore/ibge-api")

In [0]:
ibge_mongo = spark.read.parquet("/FileStore/ibge-mongo")\
                  .filter((col("sexo")=="Mulher")&(col("idade")>=20)&(col("idade")<=40))

In [0]:
ibge_api.printSchema()
ibge_api.show(truncate=False)
# UF: struct
# display(ibge_api.groupBy("nome").agg(collect_list("id")))

root
-- id: long (nullable = true)
-- microrregiao: struct (nullable = true)
 |-- id: long (nullable = true)
 |-- mesorregiao: struct (nullable = true)
 | |-- UF: struct (nullable = true)
 | | |-- id: long (nullable = true)
 | | |-- nome: string (nullable = true)
 | | |-- regiao: struct (nullable = true)
 | | | |-- id: long (nullable = true)
 | | | |-- nome: string (nullable = true)
 | | | |-- sigla: string (nullable = true)
 | | |-- sigla: string (nullable = true)
 | |-- id: long (nullable = true)
 | |-- nome: string (nullable = true)
 |-- nome: string (nullable = true)
-- nome: string (nullable = true)
-- regiao-imediata: struct (nullable = true)
 |-- id: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- regiao-intermediaria: struct (nullable = true)
 | |-- UF: struct (nullable = true)
 | | |-- id: long (nullable = true)
 | | |-- nome: string (nullable = true)
 | | |-- regiao: struct (nullable = true)
 | | | |-- id: long (nullable = true)
 | | | |-- nome: string (nullable = true)
 | | | |-- sigla: string (nullable = true)
 | | |-- sigla: string (nullable = true)
 | |-- id: long (nullable = true)
 | |-- nome: string (nullable = true)

+-------+--------------------------------------------------------------------------------------------------------------------+--------------+------------------------------------------------------------------------------------------+
id |microrregiao |nome |regiao-imediata |
+-------+--------------------------------------------------------------------------------------------------------------------+--------------+------------------------------------------------------------------------------------------+
3146255|[31008, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3102, Norte de Minas], Grão Mogol] |Padre Carvalho|[310008, Salinas, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3102, Montes Claros]] |
3146305|[31012, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3103, Jequitinhonha], Araçuaí] |Padre Paraíso |[310013, Teófilo Otoni, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3103, Teófilo Otoni]] |
3146404|[31024, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3106, Central Mineira], Três Marias] |Paineiras |[310070, Abaeté, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3113, Divinópolis]] |
3146503|[31044, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3109, Oeste de Minas], Formiga] |Pains |[310066, Formiga, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3113, Divinópolis]] |
3146552|[31004, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3102, Norte de Minas], Janaúba] |Pai Pedro |[310007, Janaúba, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3102, Montes Claros]] |
3146602|[31065, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3112, Zona da Mata], Juiz de Fora] |Paiva |[310027, Juiz de Fora, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3106, Juíz de Fora]] |
3146701|[31066, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3112, Zona da Mata], Cataguases] |Palma |[310032, Cataguases, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3106, Juíz de Fora]] |
3146750|[31014, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3103, Jequitinhonha], Almenara] |Palmópolis |[310015, Almenara, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3103, Teófilo Otoni]] |
3146909|[31027, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3107, Metropolitana de Belo Horizonte], Sete Lagoas] |Papagaios |[310068, Pará de Minas, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3113, Divinópolis]] |
3147006|[31002, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3101, Noroeste de Minas], Paracatu] |Paracatu |[310062, Patos de Minas, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3112, Patos de Minas]]|
3147105|[31029, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3107, Metropolitana de Belo Horizonte], Pará de Minas] |Pará de Minas |[310068, Pará de Minas, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3113, Divinópolis]] |
3147204|[31049, [[31, Minas Gerais, [3, Sudeste, SE], MG], 3110, Sul/Sudoeste de Minas], Alfenas] |Paraguaçu |[310042, Alfenas, [[31, Minas Gerais, [3, Sudeste, SE], MG], 310

In [0]:
ibge_mongo.printSchema()
ibge_mongo.show(truncate=False)

root
-- _id: struct (nullable = true)
 |-- oid: string (nullable = true)
-- ano: integer (nullable = true)
-- anosesco: double (nullable = true)
-- cor: string (nullable = true)
-- graduacao: string (nullable = true)
-- horastrab: double (nullable = true)
-- idade: integer (nullable = true)
-- ocup: string (nullable = true)
-- renda: double (nullable = true)
-- sexo: string (nullable = true)
-- trab: string (nullable = true)
-- trimestre: integer (nullable = true)
-- uf: string (nullable = true)

+--------------------------+----+--------+--------+---------+---------+-----+-------------------+------+------+---------------------------------+---------+----+
_id |ano |anosesco|cor |graduacao|horastrab|idade|ocup |renda |sexo |trab |trimestre|uf |
+--------------------------+----+--------+--------+---------+---------+-----+-------------------+------+------+---------------------------------+---------+----+
[600ce0685daa95c6616013e2]|2020|12.0 |Parda |null |null |20 |null |null |Mulher|Pessoas fora da força de trabalho|3 |Acre|
[600ce0685daa95c6616013ec]|2020|10.0 |Parda |null |null |22 |Pessoas desocupadas|null |Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c6616013f8]|2020|11.0 |Parda |null |4.0 |22 |Pessoas ocupadas |300.0 |Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c6616013fb]|2020|6.0 |Parda |null |null |25 |null |null |Mulher|Pessoas fora da força de trabalho|3 |Acre|
[600ce0685daa95c66160140d]|2020|11.0 |Parda |null |null |26 |Pessoas desocupadas|null |Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c661601413]|2020|12.0 |Parda |null |null |30 |null |null |Mulher|Pessoas fora da força de trabalho|3 |Acre|
[600ce0685daa95c661601415]|2020|13.0 |Parda |null |0.0 |35 |Pessoas ocupadas |1200.0|Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c661601424]|2020|9.0 |Parda |null |40.0 |37 |Pessoas ocupadas |200.0 |Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c661601427]|2020|12.0 |Branca |null |null |21 |Pessoas desocupadas|null |Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c66160142b]|2020|12.0 |Parda |null |null |21 |null |null |Mulher|Pessoas fora da força de trabalho|3 |Acre|
[600ce0685daa95c66160142d]|2020|12.0 |Parda |Não |null |21 |Pessoas desocupadas|null |Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c661601433]|2020|14.0 |Parda |Não |null |34 |null |null |Mulher|Pessoas fora da força de trabalho|3 |Acre|
[600ce0685daa95c661601437]|2020|12.0 |Parda |null |40.0 |39 |Pessoas ocupadas |1800.0|Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c661601439]|2020|16.0 |Parda |null |0.0 |33 |Pessoas ocupadas |1200.0|Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c66160143c]|2020|13.0 |Parda |Não |null |39 |Pessoas desocupadas|null |Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c661601447]|2020|12.0 |Indígena|null |2.0 |30 |Pessoas ocupadas |0.0 |Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c661601450]|2020|16.0 |Parda |null |7.0 |36 |Pessoas ocupadas |1300.0|Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c661601453]|2020|16.0 |Parda |null |40.0 |30 |Pessoas ocupadas |4500.0|Mulher|Pessoas na força de trabalho |3 |Acre|
[600ce0685daa95c66160145e]|2020|6.0 |Parda |null |null |33 |null |null |Mulher|Pessoas fora da força de trabalho|3 |Acre|
[600ce0685daa95c661601462]|2020|12.0 |Parda |null |40.0 |25 |Pessoas ocupadas |1300.0|Mulher|Pessoas na força de trabalho |3 |Acre|
+--------------------------+----+--------+--------+---------+---------+-----+-------------------+------+------+---------------------------------+---------+----+
only showing top 20 rows

**Questions to be answered after ingesting the data:**

**What is the average income of the women between 20 and 40 years old in your database?**

In [0]:
display(ibge_mongo.agg({"renda": "avg"}).select(format_number('avg(renda)', 2).alias('Average income')).collect())


Average income
"1,809.28"


**What is the average income of the women between 20 and 40 years in 'Distrito Federal'?**

In [0]:
display(ibge_mongo.filter("uf = 'Distrito Federal'").agg({"renda": "avg"}).select(format_number('avg(renda)', 2).alias('Average income in DF')).collect())

Average income in DF
"2,795.94"


**What is the average income of women between 20 and 40 years old living in the "região sudeste" southeastern region of the country?**

In [0]:
RegiaoSudeste = ibge_api.select("microrregiao.mesorregiao.UF.nome").withColumnRenamed("nome","uf").filter(col("regiao-imediata.regiao-intermediaria.UF.regiao.sigla")=="SE").groupBy("uf").count()

display(ibge_mongo.join(RegiaoSudeste, ibge_mongo.uf == RegiaoSudeste.uf).groupBy().avg('renda').select(format_number('avg(renda)', 2).alias('Average income in Southeastern')).collect())


  


Average income in Southeastern
"2,034.06"


**Which Brazilian state has the lowest average income with women between 20 and 40 years old?**

In [0]:
display(
  ibge_mongo.groupBy("uf").avg("renda").select("UF",format_number('avg(renda)', 2) \
  .alias('Average income in Southeastern')).orderBy("avg(renda)").head(1)
)

UF,Average income in Southeastern
Alagoas,"1,116.10"


**Which Brazilian state has the highest average schooling with women between 20 and 40 years old?**

In [0]:
display(
  ibge_mongo.groupBy("uf").avg("anosesco").select("UF",format_number('avg(anosesco)', 2) \
  .alias('Highest average schooling')).orderBy(desc("avg(anosesco)")).head(1)
)

UF,Highest average schooling
Distrito Federal,13.11


**What is the average level of education among women between 20 and 40 years old in Mato Grosso?**

In [0]:
display(
  ibge_mongo.groupBy("uf").avg("anosesco").select("UF",format_number('avg(anosesco)', 2) \
  .alias('Average level of education in MT')).filter(ibge_mongo.uf == 'Mato Grosso')
)

UF,Average level of education in MT
Mato Grosso,12.02


**What is the average level of education among women who live in Paraná and are between 25 and 30 years old?**

In [0]:
display(
  ibge_mongo.filter("uf=='Paraná' and idade>=25 and idade<=30") \
  .groupBy("uf").avg("anosesco").select("UF",format_number('avg(anosesco)', 2).alias('Average level of education in PR')) 
)

UF,Average level of education in PR
Paraná,12.61


**What is the average income for people in the South of the country (Sul), who are in the workforce and are between 25 and 35 years old?**

In [0]:
RegiaoSul = ibge_api.select("microrregiao.mesorregiao.UF.nome").withColumnRenamed("nome","uf").filter(col("regiao-imediata.regiao-intermediaria.UF.regiao.sigla")=="S").groupBy("uf").count()

display(ibge_mongo.filter("idade>=25 and idade<=35 and trab=='Pessoas na força de trabalho'").join(RegiaoSul, ibge_mongo.uf == RegiaoSul.uf).groupBy().avg('renda').select(format_number('avg(renda)', 2).alias('Average income in South')).collect())

Average income in South
"2,231.89"


**How many mesoregion are in the state of MG (Minas Gerais)?**

In [0]:
MesoMG = ibge_api.select("microrregiao.mesorregiao.nome") \
.filter(col("microrregiao.mesorregiao.UF.nome")=="Minas Gerais")\
.groupBy("nome").count().count() 

#MesoMG

MesoMG



Out[377]: 12

**What is the average income of women residing in the North of Brazil, graduated, aged between 25 and 35 years and are black or brown?**

In [0]:
RegiaoNorte = ibge_api.select("microrregiao.mesorregiao.UF.nome").withColumnRenamed("nome","uf").filter(col("regiao-imediata.regiao-intermediaria.UF.regiao.sigla")=="N").groupBy("uf").count()

display(ibge_mongo.filter("idade>=25 and idade<=35 and (cor=='Preta' or cor=='Parda') and graduacao = 'Sim' and  trab = 'Pessoas na força de trabalho'").join(RegiaoNorte, ibge_mongo.uf == RegiaoNorte.uf).groupBy().avg('renda').select(format_number('avg(renda)', 2).alias('Average income in North')).collect())

Average income in North
"1,391.67"
